<a href="https://colab.research.google.com/github/RoyElkabetz/Tensor-Networks-Simple-Update/blob/main/univeral_tensor_network_paper__reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## uncomment only if running from google.colab
# clone the git reposetory
!git clone https://github.com/RoyElkabetz/Tensor-Networks-Simple-Update
# add path to .py files for import
import sys
sys.path.insert(1, "/content/Tensor-Networks-Simple-Update/src")

Cloning into 'Tensor-Networks-Simple-Update'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 126 (delta 61), reused 91 (delta 35), pack-reused 0
Receiving objects: 100% (126/126), 56.00 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [3]:
import numpy as np
from TensorNetwork import TensorNetwork
import SimpleUpdate as su
np.random.seed(216)


# MPS Tensor Network
mps_structure_matrix = np.array([[1, 2],
                                 [1, 2]])
# MPS Tensor Network
peps_structure_matrix = np.array([[1, 2, 3, 4, 0, 0, 0, 0],
                                  [1, 2, 0, 0, 3, 4, 0, 0],
                                  [0, 0, 1, 2, 0, 0, 3, 4,],
                                  [0, 0, 0, 0, 1, 2, 3, 4]])

star_structure_matrix = np.array([[1, 2, 3, 0, 0, 0, 0, 0, 0],
                                 [1, 0, 0, 2, 3, 0, 0, 0, 0],
                                 [0, 1, 0, 2, 0, 3, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1, 2, 3, 0],
                                 [0, 0, 0, 0, 1, 0, 2, 0, 3],
                                 [0, 0, 1, 0, 0, 0, 0, 2, 3]])

cube_structure_matrix = np.array([[1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 [1, 2, 0, 0, 0, 0, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 [0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 5, 6, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 5, 6, 0, 0],
                                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 4, 5, 6],
                                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 3, 4, 0, 0, 5, 6]])






In [8]:
# prepare simple-update parameters
pauli_x = np.array([[0, 1],
                    [1, 0]])
pauli_y = np.array([[0, -1j],
                    [1j, 0]])
pauli_z = np.array([[1, 0],
                    [0, -1]])
dts = [0.1, 0.01, 0.001, 0.0001, 0.00001]
h_k = 0.
s_i = [pauli_x / 2., pauli_y / 2., pauli_z / 2.]
s_j = [pauli_x / 2., pauli_y / 2., pauli_z / 2.]
s_k = [pauli_x / 2.]
d_max = 2


mps = TensorNetwork(structure_matrix=mps_structure_matrix, virtual_size=3)
peps = TensorNetwork(structure_matrix=peps_structure_matrix, virtual_size=3)
star = TensorNetwork(structure_matrix=star_structure_matrix, virtual_size=3)
cube = TensorNetwork(structure_matrix=cube_structure_matrix, virtual_size=3)
j_ij_mps = [-1.] * len(mps.weights)
j_ij_peps = [-1.] * len(peps.weights)
j_ij_star = [1.] * len(star.weights)
j_ij_cube = [-1.] * len(cube.weights)

In [9]:
for dt in dts:
    for _ in range(50):
        su.simple_update(tensor_network=mps, dt=dt, j_ij=j_ij_mps, h_k=h_k, s_i=s_i, s_j=s_j, s_ik=s_k, s_jk=s_k, d_max=d_max)
tensors = mps.tensors
weights = mps.weights
energy_mps = su.energy_per_site(tensors, weights, mps_structure_matrix, j_ij_mps, h_k, s_i, s_j, s_k, s_k)
print(f'mps energy: {energy_mps}, paper: {-0.44304}')

for dt in dts:
    for _ in range(50):
        su.simple_update(tensor_network=star, dt=dt, j_ij=j_ij_star, h_k=h_k, s_i=s_i, s_j=s_j, s_ik=s_k, s_jk=s_k, d_max=d_max)
tensors = star.tensors
weights = star.weights
energy_star = su.energy_per_site(tensors, weights, star_structure_matrix, j_ij_star, h_k, s_i, s_j, s_k, s_k)
print(f'star energy: {energy_star}, paper: {-0.37523}')

for dt in dts:
    for _ in range(50):
        su.simple_update(tensor_network=cube, dt=dt, j_ij=j_ij_cube, h_k=h_k, s_i=s_i, s_j=s_j, s_ik=s_k, s_jk=s_k, d_max=d_max)
tensors = cube.tensors
weights = cube.weights
energy_cube = su.energy_per_site(tensors, weights, cube_structure_matrix, j_ij_cube, h_k, s_i, s_j, s_k, s_k)
print(f'cube energy: {energy_cube}, paper: {-0.89253}')

mps energy: (-0.42764890032369+0j), paper: -0.44304
star energy: (-0.3749985308498954+0j), paper: -0.37523
cube energy: (-0.8915746895610899+0j), paper: -0.89253
